In [1]:
import gym
from gym import wrappers

# env = gym.make('CartPole-v0')
# env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

# total_reward = 0

# for i_episode in range(3):
#     observation = env.reset()
#     for t in range(1000000):
#         env.render()
# #         if t % 50 == 0:
# #         print(observation)
#         if observation[3] > 0:
#             action = 1
#         elif observation[3] < 0:
#             action = 0
#         if abs(observation[3]) < .15 and abs(observation[1]) > .5:
#             action = 0 if (action == 1) else 1

#         observation, reward, done, info = env.step(action)
#         total_reward += reward
# #         print done, reward
#         if done:
#             print info
# #             print(observation)
#             print("Episode finished after {} timesteps".format(t+1))
#             break

# print("average reward is {}".format(total_reward / 100))

### CartPole with GA

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense


def get_model():
    model = Sequential()
    model.add(Dense(4, input_shape=[4], activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(6, activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(2, activation='tanh', kernel_initializer='random_uniform'))
    return model

Using TensorFlow backend.


In [3]:
# env = gym.make('CartPoleU-v0')
gym.envs.register(
    id='CartPoleUV-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    max_episode_steps=5000,      # MountainCar-v0 uses 200
)

In [4]:
env = gym.make('CartPoleUV-v0')

[2017-12-19 17:58:05,739] Making new env: CartPoleUV-v0


In [5]:
def get_action_from_tanh(value):
    return 1 if value[0][0]>value[0][1] else 0
    

In [6]:
import gym
import random
import numpy as np
from gym import wrappers



env = gym.make('CartPoleUV-v0')
env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

def start_epoch(population, ):
    for agent in population:
    
        reward_boost = 1
        reward_boost_scaler = 2
        
        max_angle = 0
        pos_cicles = 0
        neg_cicles = 0
        
        observation = env.reset()
        agent.total_reward = 0

        while True:
            env.render()

            action = get_action_from_tanh(
                agent.predict(
                    # v1 with only 2 observations. Without convergence
#                     np.array([observation[3], observation[1]]).reshape([1, 2])
                    # v2 with all 4 observations. convergence.
                    np.array(observation).reshape([1, 4])
                )
            )
            #observation[1] angle of plank
#             print observation[1] 
            observation, reward, done, info = env.step(action)
            
            # reward boost computation         
            if observation[1] > 0:
                pos_cicles += 1
                if neg_cicles <= 2:
                    neg_cicles = 0
            else:
                neg_cicles += 1
                if pos_cicles <= 2:
                    pos_cicles = 0 
            
            if neg_cicles > 2 and pos_cicles > 2:
                print 'agent boosted for balancing attempt'
                reward_boost *= reward_boost_scaler + (10/((neg_cicles + pos_cicles)/2))
                pos_cicles, neg_cicles = 0, 0
            
            max_angle = abs(observation[1]) if abs(observation[1]) > max_angle else max_angle        
                    
            agent.total_reward += reward * reward_boost
        
            if done:
                
                if max_angle < 0.4:
                    print 'agent boosted for low max_angles'
#                     agent.total_reward *= 3
    #             print(observation)
#                 print("Episode finished after {} timesteps".format(t+1))
                break
    return population


[2017-12-19 17:58:08,369] Making new env: CartPoleUV-v0
[2017-12-19 17:58:08,381] Creating monitor directory /tmp/cartpole-experiment-1


In [7]:
def selecting(population, num_of_selected = 2):
    return sorted(weighted_population, key=lambda k: k.total_reward, reverse=True) 

In [8]:
def crossover(agent1, agent2):
    a1_w = agent1.get_weights()
    a2_w = agent2.get_weights()
    
    a1_w_n, a2_w_n = [], []

    for i, w in enumerate(a1_w):

#         if not len(a1_w)-1 == i:
        res_shape = w.shape
        w = w.flatten()
        a2_w[i] = a2_w[i].flatten()

        gen_separator = random.randint(1, w.shape[0])

        a1_w_n.append(np.concatenate([w[0:gen_separator], a2_w[i][gen_separator:]]).reshape(res_shape))
        a2_w_n.append(np.concatenate([a2_w[i][0:gen_separator], w[gen_separator:]]).reshape(res_shape))
#         else:
#             a1_w_n.append(random.choice([a1_w[i], a2_w[i]]))
#             a2_w_n.append(random.choice([a1_w[i], a2_w[i]]))
    
    return a1_w_n, a2_w_n


def mutation(agent):
    weights = agent.get_weights()
    for i, w in enumerate(weights):
#         weights[i] =  w * ((np.random.rand(*w.shape) - 0.5) * 3 + (np.random.rand(*w.shape) - 0.5))
        weights[i] =  w * ((np.random.rand(*w.shape) - 0.5)/100)
    return weights

In [9]:
def regenerate_population():
    return [get_model() for _ in range(10)]

In [ ]:
# Agents creation
population = regenerate_population()

best_agent = population[0]
best_agent.total_reward = -9999

p = 1
    
while True:
    print "Population: ", p
    weighted_population = start_epoch(population)
    population_sorted_by_results = selecting(weighted_population)
    
    if best_agent.total_reward <= population_sorted_by_results[0].total_reward:
        best_agent = population_sorted_by_results[0]
   
    if population_sorted_by_results[0].total_reward < 200 or max(env.get_episode_lengths()) < 200:
        print 'New population was created'
        new_population = regenerate_population()
    else:
        
        new_population = population_sorted_by_results
        print ('Same population: reward-',
                population_sorted_by_results[0].total_reward,
                max(env.get_episode_lengths()))
        
        # Copy 4th winners #1-4
        for a in range(4):
            new_population[a].set_weights(population_sorted_by_results[a].get_weights())

        # Add 1 child of 2 winners #5
        childs_of_winners = crossover(population_sorted_by_results[0], population_sorted_by_results[1])
        new_population[4].set_weights(random.choice(childs_of_winners))

        # Add 3 child of random 2 winners from 4 of them #6-10
        for i in range(5,10):
            parent_a = 0
            parent_b = parent_a

            while parent_a==parent_b:
                parent_a = random.choice([0, 1, 2, 3])
                parent_b = random.choice([0, 1, 2, 3])

            childs = crossover(population_sorted_by_results[parent_a], population_sorted_by_results[parent_b])
            new_population[i].set_weights(random.choice(childs))

        for agent in new_population[4:]:
            agent.set_weights(mutation(agent))

    population = new_population
    p += 1
    print len(population)

[2017-12-19 17:58:24,619] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.1953.video000000.mp4


Population:  1


[2017-12-19 17:58:35,231] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.1953.video000001.mp4


agent boosted for balancing attempt


[2017-12-19 17:58:37,311] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.1953.video000008.mp4


agent boosted for low max_angles
New population was created
10
Population:  2
agent boosted for low max_angles
New population was created
10
Population:  3
agent boosted for low max_angles


[2017-12-19 17:58:58,903] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.1953.video000027.mp4


agent boosted for low max_angles
New population was created
10
Population:  4
agent boosted for low max_angles
agent boosted for low max_angles
New population was created
10
Population:  5
agent boosted for balancing attempt
agent boosted for low max_angles


In [ ]:
obsevation

In [ ]:
max(env.get_episode_lengths())

In [ ]:
population_sorted_by_results[0].total_reward

In [ ]:
population_sorted_by_results[-i-1].set_weights(population_sorted_by_results[-i-1].get_weights())

In [ ]:
best_agent.total_reward

In [ ]:
new_population

In [ ]:
population_sorted_by_results[a]

In [ ]:
best_agent.total_reward

In [ ]:
best_agent.total_reward